# libs & configs

In [43]:

import pandas as pd
from pandas import to_datetime as pdDatetime
from datetime import datetime
from pandas import to_numeric as pdNumeric
import os
import sys
from collections import defaultdict

In [44]:
# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

In [45]:
from Scripts.functions import GetAno

# 00 - juntando das informações infosfot x Bling

## clientes °

In [54]:
# -- 1 lendo arquivos
df_clients_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_bling.xlsx")
df_clients_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_infosoft.xlsx")

# -- 2 - padroniza os nomes para maior facilidade
df_clients_infosoft.rename(columns={'Cgc Cpf':'Clie Cgc Cpf'}, 
                                inplace=True)
df_clients_bling.rename(columns={'CPF/CNPJ': 'Clie Cgc Cpf',
                                  'Nome':'Descrição', 
                                  'Endereço': 'Endereco',
                                  'UF':'Cd Uf', 
                                  'Cidade':'Nm Cidade', 
                                  'Bairro':'Bairro',
                                'Cliente desde':'Dt Cadastro', 
                                'Email':'E Mail'
                                }, inplace=True)

# -- 3 - faz o full join 
df_clientes_merge = pd.merge(df_clients_infosoft,df_clients_bling,
                                  on='Clie Cgc Cpf',   # chave primaria
                                  how='outer',    # full outer join no pandas       
                                  suffixes=('_infosoft','_bling')) # renomeias o sufixo das colunas (col1_df1,col2_df2...)

# -- 4 - filtrando para todos os clients ou seja '''full join com priorização do df1'''
cols = ['Descrição', 'Endereco', 'Cd Uf','Nm Cidade', 'Bairro', 'Dt Cadastro','E Mail'] # colunas que quero manter (na esquerda)
for col in cols: 
    df_clientes_merge[col] = df_clientes_merge[f'{col}_infosoft'].combine_first(df_clientes_merge[f'{col}_bling']) # where quando a esquerda for vazia colca conteudo da direita, colca tudo na nova coluna 
    
df_clientes_masquerade = df_clientes_merge[['Clie Cgc Cpf'] + cols] # atribui a df final o conteudo completo nas colunas novas sem esquecer da chave primeira



## nfs ~


In [55]:
'''
corrigir cod vendedores bling
nr de pedido

'''


# -- 1 - le os arquivos
df_nfs_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_bling.xlsx")
df_nfs_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_infosoft.xlsx")



# -- 2 - padroniza as colunas para evitar complicações
df_nfs_bling.rename(columns={'Data de emissão':'Dt. emissão', 
                             'Data de Saída/Entrada':'Dt Sai Ent', 
                             'Número':'Nr. nota', 
                             'CNPJ/CPF':'Clie Cgc Cpf',
                             'Código do vendedor':'Cd Vendedor', 
                             'Desconto':'Vl Total Desc', 
                             'Valor total líquido':'Vl Total Nota'
                             },inplace=True)

# -- 3 -  junta as duas df 
df_nfs_masquerade = pd.concat([df_nfs_bling,df_nfs_infosoft], axis=0)

# -- 4 - arrumando vendedores 

# -- 5 - padroniza os tipos
df_nfs_masquerade['Dt Sai Ent'] = pdDatetime(df_nfs_masquerade['Dt Sai Ent'])
df_nfs_masquerade['Dt. emissão'] = pdDatetime(df_nfs_masquerade['Dt. emissão'])
df_nfs_masquerade['Vl Total Desc'] = df_nfs_masquerade['Vl Total Desc'].str.replace(',','')
df_nfs_masquerade['Vl Total Nota'] = df_nfs_masquerade['Vl Total Nota'].str.replace(',','')
df_nfs_masquerade['Vl Total Desc'] = pdNumeric(df_nfs_masquerade['Vl Total Desc'],errors='coerce')
df_nfs_masquerade['Vl Total Nota'] =  pdNumeric(df_nfs_masquerade['Vl Total Nota'],errors='coerce')
df_nfs_masquerade['Nr Pedido'] =  pdNumeric(df_nfs_masquerade['Nr Pedido'],errors='coerce')

## pedidos

In [56]:
df_pedidos_masquerade = pd.read_excel(r"../Data/01-01-23_08-07-25/sh-pedidos_infosoft.xlsx")
df_pedidos_masquerade.rename(columns={'CNPJ - CPF':'Clie Cgc Cpf', 
                                      'Vl. total': 'Vl venda',
                                      'Nr. pedido': 'Nr Pedido'
                                      },inplace=True)

df_pedidos_masquerade['Nr Pedido'] = pdNumeric(df_pedidos_masquerade['Nr Pedido'])
df_pedidos_masquerade.columns

Index(['Nr Pedido', 'Dt. pedido', 'Clie Cgc Cpf',
       'Pedido de venda - Vl. desconto', 'Vl venda'],
      dtype='object')

In [57]:
df_nfs_masquerade.columns


Index(['Dt. emissão', 'Dt Sai Ent', 'Nr Pedido', 'Nr. nota', 'Clie Cgc Cpf',
       'Cd Vendedor', 'Vl Total Desc', 'Vl Total Nota'],
      dtype='object')

## produtos

## vendedores °

In [58]:
df_vendedores_masquerade = pd.read_excel(r'..\Data\01-01-23_08-07-25\sh-vendedores_infosoft.xlsx')
df_vendedores_masquerade = df_vendedores_masquerade[~pd.isna(df_vendedores_masquerade['Nm Vendedor'])]
df_vendedores_masquerade['Cd Vendedor'] = df_vendedores_masquerade['Cd Vendedor'].astype(int) 

df_vendedores_masquerade

,Nm supervisor,Cd supervisor,Cd Vendedor,Nm Vendedor,Endereco,Cd Uf,Ds Regiao,Bairro,Dt Cadastro,E Mail
1,ALDO,1.0,1,ALDO,NaN,NaN,NORTE,NaN,NaT,acz7@terra.com.br
4,FLAVIO E LILIAN,2.0,2,FLAVIO E LILIAN,NaN,NaN,NaN,NaN,2019-03-13,flavioelilian@hotmail.com
7,NaN,NaN,3,AV,NaN,NaN,NaN,NaN,2019-03-13,NaN
10,NaN,NaN,4,GERARDO,NaN,NaN,NaN,NaN,2019-03-13,NaN
13,NaN,NaN,5,ARLEY,NaN,NaN,NaN,NaN,2019-03-13,NaN
...,...,...,...,...,...,...,...,...,...,...
319,NaN,NaN,107,JOÃO PAULO RIO DE JANEIRO,NaN,NaN,NaN,NaN,2025-01-15,NaN
322,NaN,NaN,108,PAULO RIO DE JAINEIRO,NaN,NaN,NaN,NaN,2025-01-27,NaN
325,NaN,NaN,109,CAIO MARCELO VIGHI VISCARD E CIA LT,NaN,NaN,NaN,NaN,2025-06-25,NaN
328,ANDREIA,64.0,110,AMERIPEL CARLOS,NaN,NaN,NaN,NaN,2025-06-26,NaN



# 15 - faturamento completo mes ano, por "produto"



In [ ]:
''' mes / papelaria / mer_livre / sapatilha / eventos / modinha / puerilcultura / total 

total: / ....... /'''

##  16 - faturameno completo mes indivdual separado (modelo)



In [ ]:
'''
mes 
papelaria 
 mer_livre 
sapatilha 
eventos 
 modinha 
puerilcultura 
total 
'''

In [ ]:
# 1 preenchendo o modelo


# 05 - resumo de venda/faturamento - fechamento anual


In [ ]:
''' mes-ano / vendas / faturado 
total / .../ '''

In [ ]:
# -- 1 - defines as bases de dados e trata certas colunas
df_nfs = {
    
    'data': ['22/02/2025', '22/02/2024','22/02/2024','22/04/2025','22/05/2025'],
    'valor': [1,2,3,4,5]
}
df_nfs = pd.DataFrame(df_nfs)
df_nfs['data'] = pd.to_datetime(df_nfs['data'])
# df_pedidos = pd.read_excel()

# -- 2 -  defines as colunas do DataFrame
colunas = ['mes-ano', 'vendas', 'faturamento']    
df_faturamento_mes_ano = pd.DataFrame(columns=colunas)

# -- 3 - pega os anos
anos = [2022, 2023, 2024, 2025]

# -- 4 - se o ano for valido, filtra por mes e atribui ao df principal
dfs_por_ano = defaultdict(dict)
for ano in anos: # posso usar df_nfs['data'].dt.year.unique() // que eu pego todos os anos da planilha
    for mes in df_nfs['data'].dt.month.unique(): # epgo apenas os meses que exitem na planilha
        df_nfs_mes = df_nfs[ # filtro
            (df_nfs['data'].dt.year == ano) &
            (df_nfs['data'].dt.month == mes) 
            ] 
        if not df_nfs_mes.empty: # ingnoro se estiver vazio
            total = df_nfs_mes['valor'].sum()
            dfs_por_ano[ano][mes] = total
            

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3392\1127894070.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_nfs['data'] = pd.to_datetime(df_nfs['data'])


# 09 - comissões por periodos



In [59]:
''' 
supervisor
nm_vendedor
data / nf / cliente / venda / faturado / aliquota / comissão / cliente_nv / total
nm_vendedor / faturado
3% supervisor / subtotal
total / 
'''

' \nsupervisor\nnm_vendedor\ndata / nf / cliente / venda / faturado / aliquota / comissão / cliente_nv / total\nnm_vendedor / faturado\n3% supervisor / subtotal\ntotal / \n'

In [60]:
# -- 1 - defino as coluas de nossa df bruta e trago as df necessrias para o relatorio 
df_comission_raw = pd.DataFrame(columns=['Clie Cgc Cpf','Dt. emissão', 'Nr. nota', 'Cd Vendedor', 'Vl Total Nota', 'aliquota', 'comissao', 'total'])
df_vendedores = df_vendedores_masquerade.copy() # para pegar o vendedor
df_clientes = df_clientes_masquerade.copy() # para pegar o cliente
df_pedidos = df_pedidos_masquerade.copy() # para pegar os pedidos
df_nfs = df_nfs_masquerade[['Dt. emissão', 'Nr. nota','Nr Pedido', 'Clie Cgc Cpf', 'Cd Vendedor', 'Vl Total Nota']]

# -- 1 - atribuindo infos das nfs 
df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)

# -- 2 - defininido o periodo (mes e ano) e filtrando nossa df
ano = 2025
mes = 6
df_comission_raw = df_comission_raw[(df_comission_raw['Dt. emissão'].dt.year == ano) & 
                (df_comission_raw['Dt. emissão'].dt.month == mes)]

# -- 3 - fazer um semi join (pega apegas nfs que tivrem com cd vendedor) - por preucação
df_comission_raw = df_comission_raw[df_comission_raw['Cd Vendedor'].isin(df_vendedores['Cd Vendedor'])] #  pandas não tem suporte direto para semi join o jeito a epanas filtrar oque e muito pratico!

# -- 4 - fazer merge para epgar o nome dos vendedores pelo cod, posteriormente pegar o supervisor e aliquota tambem 
df_comission_raw['aliquota'] = 7 # temporario
df_comission_raw = df_comission_raw.merge(df_vendedores[['Cd Vendedor', 'Nm Vendedor','Nm supervisor']], 
                             on='Cd Vendedor', how='left')

# -- 5 - fazer merge para pegar o nome do cliente (left join em duas colunas basicamente..)
df_comission_raw = df_comission_raw.merge(df_clientes[['Clie Cgc Cpf', 'Descrição']],
                      on='Clie Cgc Cpf', how='left')

# -- 6 - merge para pegar o valor do pedido
df_comission_raw = df_comission_raw.merge(df_pedidos[['Nr Pedido', 'Vl venda']],
                      on='Nr Pedido', how='left')

# -- 7 -  definindo comissão a ser recebida 
df_comission_raw['comissao'] = df_comission_raw['Vl Total Nota']*(df_comission_raw['aliquota']/100)

# -- 8 - definindo clientes novos
df_clientes_novos = df_clientes[df_clientes['Dt Cadastro'] > datetime(ano, mes, 1)]
df_clientes_novos['cliente_nv'] = 20

# -- 9 - fazendo merge para atribuir os clientes novos 
df_comission_raw = df_comission_raw.merge(df_clientes_novos[['Clie Cgc Cpf', 'cliente_nv']],
                      on='Clie Cgc Cpf', how='left', suffixes=('_raw','_new'))

# -- 10 - definindo comissão total 
df_comission_raw['total'] = df_comission_raw['cliente_nv'].fillna(0)+df_comission_raw['comissao'].fillna(0)

# --  - atribuindo informações a nossa df bruta


C:\Users\kwesz\AppData\Local\Temp\ipykernel_10172\2796649900.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)
C:\Users\kwesz\AppData\Local\Temp\ipykernel_10172\2796649900.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clientes_novos['cliente_nv'] = 20


In [61]:
df_comission = df_comission_raw[['Nm supervisor', 'Nm Vendedor','Dt. emissão', 'Nr. nota','Descrição','Vl venda', 'Vl Total Nota','aliquota','comissao','cliente_nv','total']]

